<a href="https://colab.research.google.com/github/KarlMh/projects/blob/main/fcc_book_recommendation_knn_karlmh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-08-13 12:31:24--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  85.2MB/s    in 0.3s    

2024-08-13 12:31:25 (85.2 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Check the initial size of dataframes
print(f"Initial df_ratings shape: {df_ratings.shape}")
print(f"Initial df_books shape: {df_books.shape}")

# Filter users with at least 20 ratings
user_counts = df_ratings['user'].value_counts()
users_to_keep = user_counts[user_counts >= 1].index
df_ratings_filtered = df_ratings[df_ratings['user'].isin(users_to_keep)]

# Filter books with at least 20 ratings
book_counts = df_ratings_filtered['isbn'].value_counts()
books_to_keep = book_counts[book_counts >= 1].index
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(books_to_keep)]

# Check the size of the filtered dataframe
print(f"Filtered df_ratings shape: {df_ratings_filtered.shape}")

# Merge the ratings with book titles
df = pd.merge(df_ratings_filtered, df_books, on='isbn')
print(f"Merged dataframe shape: {df.shape}")

# Create a pivot table
try:
    book_user_matrix = df.pivot_table(index='title', columns='user', values='rating').fillna(0)
    print(f"Pivot table shape: {book_user_matrix.shape}")
except Exception as e:
    print(f"Error creating pivot table: {e}")

# Create a sparse matrix
book_user_sparse_matrix = csr_matrix(book_user_matrix.values)
print(f"Sparse matrix shape: {book_user_sparse_matrix.shape}")

# Fit the KNN model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_user_sparse_matrix)

Initial df_ratings shape: (1149780, 3)
Initial df_books shape: (271379, 3)
Filtered df_ratings shape: (1149780, 3)
Merged dataframe shape: (1031175, 5)


<ipython-input-4-ca69e59be300>:24: PerformanceWarning: The following operation may generate 22206076630 cells in the resulting pandas object.
  book_user_matrix = df.pivot_table(index='title', columns='user', values='rating').fillna(0)


In [ ]:
# Create the get_recommends function
def get_recommends(book_title, n_recommendations=5):
    if book_title not in book_user_matrix.index:
        raise ValueError(f"Book '{book_title}' not found in the dataset.")

    # Get the index of the book
    book_index = book_user_matrix.index.get_loc(book_title)

    # Get the distances and indices of the nearest neighbors
    distances, indices = model.kneighbors(book_user_sparse_matrix[book_index].reshape(1, -1), n_neighbors=n_recommendations + 1)

    # Get the titles and distances of the recommended books
    recommended_books = []
    for i in range(1, len(distances.flatten())):  # Skip the first one because it’s the input book itself
        book_idx = indices.flatten()[i]
        recommended_books.append([book_user_matrix.index[book_idx], distances.flatten()[i]])

    return [book_title, recommended_books]

get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

In [ ]:

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()